In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.!!!!')


Libraries imported.!!!!


After importing all standard libraries and modules , we import BEatiful Soup for scraping Wikipedia page for Toronto. (as from Turkey we can not reach Wikipedia (dont ask me why) we reach WikiZero and use that url .

In [2]:
from bs4 import BeautifulSoup
print("Beatiful Soup Imported")
!pip install lxml
print("lxml installed ")
!wget -q -O 'html_doc_toronto.html' http://www.wikizero.biz/index.php?q=aHR0cHM6Ly9lbi53aWtpcGVkaWEub3JnL3dpa2kvTGlzdF9vZl9wb3N0YWxfY29kZXNfb2ZfQ2FuYWRhOl9N
with open('html_doc_toronto.html') as fp:
    soup = BeautifulSoup(fp)


Beatiful Soup Imported
lxml installed 


We find 'table' by using soup.find 
Th_tags id from finding Table headings  which create Column Names by using get_text function
We define a DataFrame named dfToronto to put all data into.

In [3]:
table = soup.find( "table", {"class":"wikitable sortable"}) 
rows=list()
column_names = [] 
n_columns = 0
n_rows=0
th_tags = table.find_all('th')
tr_tags = table.find_all('tr')
for th in th_tags:
    tag=str(th.get_text())
    #print(tag,type(tag))
    tag=tag.replace("\n","")
    column_names.append(tag)
    
    #column_names.append(th.get_text())
    
columns = column_names if len(column_names) > 0 else range(0,n_columns)
for row in table.find_all('tr'):
    
     # Determine the number of rows in the table
    td_tags = row.find_all('td')
    
    if len(td_tags) > 0:
        n_rows+=1
        if n_columns == 0:
                        # Set the number of columns for our table
            n_columns = len(td_tags)
    

dfToronto = pd.DataFrame(columns = columns,
                        index= range(0,n_rows))
dfToronto_new = pd.DataFrame(columns = columns,
                        index= range(0,n_rows))
print("Toronto DataFrame with Not Assigned Data   shape : ",dfToronto.shape)

Toronto DataFrame with Not Assigned Data   shape :  (288, 3)


For each tr (table record) in the Table 
    Find Td (table data) and put every TD to Toronto DataFrame Columns(use .iat)
    
    
    
    

In [4]:
row_marker=0
column_marker=0
for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')   
    for column in columns:
        tagd=column.get_text()
        #print(tagd,type(tagd))
        tagd=tagd.replace("\n","")
        dfToronto.iat[row_marker,column_marker] = tagd
        #dfToronto.iat[row_marker,column_marker] = column.get_text()
        column_marker += 1
        
    if len(columns) > 0:
        row_marker += 1
print("Toronto w/o  Aggregation shape : ",dfToronto_new.shape)

Toronto w/o  Aggregation shape :  (288, 3)


Cretae a new DataFrame dfToronto_new  with 
    If Neighborhood is 'not Assigned' make Neighborhhod same as Borough
    If Borough is Not Assigned skip that row !

In [5]:
dfToronto_new=dfToronto[dfToronto.Borough != 'Not assigned']
dfToronto_new['Neighbourhood'] = np.where(dfToronto_new['Neighbourhood'] == 'Not assigned', 
                                            dfToronto_new['Borough'], dfToronto_new['Neighbourhood'])
print(dfToronto_new.shape)

(211, 3)


/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


To Aggregate Neighborhoods Let's group by DataFrame PostCode and Borough and use Aggregate function to concanate the Neighborhood data by using lambda function
dfToronto_newer is the final DataFrame

In [6]:
unionofneigh = lambda a: ",".join(a)

dfToronto_newer=dfToronto_new.groupby(['Postcode','Borough'],as_index=False).agg({'Neighbourhood': unionofneigh})
print("Toronto w/o  Aggregation shape : ",dfToronto_new.shape)
print("Toronto with Aggregation shape : ",dfToronto_newer.shape)
dfToronto_newer.rename(columns={"Postcode": "Postal Code"})
dfToronto_newer.columns

Toronto w/o  Aggregation shape :  (211, 3)
Toronto with Aggregation shape :  (103, 3)


Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

To add longitude and latitude use csv file .


In [7]:
url='http://cocl.us/Geospatial_data' 
dfToronto_LatLong=pd.read_csv(url)
dfToronto_LatLong.shape
dfToronto_LatLong.rename(index=str,columns={"Postal Code": "Postcode"})
dfToronto_LatLong.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

In [10]:
dfToronto_merge=pd.merge(dfToronto_newer,dfToronto_LatLong,left_on='Postcode',
                         right_on='Postal Code',how='left')
print(dfToronto_merge.columns)
dfToronto_merge.drop(columns='Postal Code',axis=1)
dfToronto_new_merge=dfToronto_merge.filter(['Postcode', 'Borough', 'Neighbourhood', 'Latitude',
       'Longitude'],axis=1)
dfToronto_new_merge
dfToronto_new_merge.shape

Index(['Postcode', 'Borough', 'Neighbourhood', 'Postal Code', 'Latitude',
       'Longitude'],
      dtype='object')


(103, 5)